# Data cleaning project
## This project uses a kaggle dataset: mohammedarfathr/smartwatch-health-data-uncleaned.

In [1]:
import kaggle
import os
    
# Set the path to the download directory
download_path = './data'
    
# Ensure the download directory exists
os.makedirs(download_path, exist_ok=True)
    
# Download the dataset
kaggle.api.authenticate()
kaggle.api.dataset_download_files('mohammedarfathr/smartwatch-health-data-uncleaned', path=download_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/mohammedarfathr/smartwatch-health-data-uncleaned


In [120]:
import pandas as pd
import numpy as np
df = pd.read_csv(f"{download_path}/unclean_smartwatch_health_data.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User ID                 9799 non-null   float64
 1   Heart Rate (BPM)        9600 non-null   float64
 2   Blood Oxygen Level (%)  9700 non-null   float64
 3   Step Count              9900 non-null   float64
 4   Sleep Duration (hours)  9850 non-null   object 
 5   Activity Level          9800 non-null   object 
 6   Stress Level            9800 non-null   object 
dtypes: float64(4), object(3)
memory usage: 547.0+ KB


### Let's first do an overview of the data.

In [ ]:
df.info()

In [121]:
df.shape

(10000, 7)

In [138]:
df.head(20)

,User ID,Heart Rate (BPM),Blood Oxygen Level (%),Step Count,Sleep Duration (hours),Activity Level,Stress Level
0,4174.0,58.9,98.8,5450,7.2,Highly_Active,1
1,NaN,NaN,98.5,727,6.5,Highly_Active,5
2,1860.0,247.8,97.1,2826,0.0,Highly_Active,5
3,2294.0,40.0,96.9,13797,7.4,Active,3
4,2130.0,62.0,98.6,15679,NaN,Highly_Active,6
5,2095.0,96.3,94.2,10205,8.4,Highly_Active,10
6,4772.0,47.3,95.4,3208,7.9,Sedentary,2
7,4092.0,81.7,96.0,6051,5.2,Sedentary,1
8,3979.0,93.3,97.2,1775,5.5,Sedentary,1
9,3169.0,51.4,97.3,7940,8.0,Sedentary,8


In [6]:
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User ID                 9799 non-null   float64
 1   Heart Rate (BPM)        9600 non-null   float64
 2   Blood Oxygen Level (%)  9700 non-null   float64
 3   Step Count              9900 non-null   float64
 4   Sleep Duration (hours)  9850 non-null   object 
 5   Activity Level          9800 non-null   object 
 6   Stress Level            9800 non-null   object 
dtypes: float64(4), object(3)
memory usage: 547.0+ KB


In [110]:
# Let's check for duplicates
df.duplicated().sum()

0

#### We can see that there are no duplicates, but there are some typos and missing values. 
#### We can also see that the values have a lot of decimal places.
#### Let's now proceed to cleaning the data.

#### ================
#### 'User ID' column cleanup
#### ================

In [155]:
# Let's fill unique ids for missing values in User ID column
def generate_unique_ids_for_nan(df, column_name, start_range, end_range):
    """
    Generates unique IDs within a specified range for NaN values in a DataFrame column.

    Args:
        df (pd.DataFrame): The DataFrame to modify.
        column_name (str): The name of the column containing NaN values.
        start_range (int): The starting value for the unique ID range.
        end_range (int): The ending value for the unique ID range (inclusive).

    Returns:
        pd.DataFrame: The modified DataFrame with NaN values replaced by unique IDs.
    """
    # Create a set of existing values in the column to ensure uniqueness
    existing_ids = set(df[column_name].dropna())
    
    # Generate unique IDs for NaN values
    num_nan = df[column_name].isnull().sum()
    
    # Generate a sequence of numbers within the range
    id_sequence = list(range(start_range, end_range + 1))
    
    # Filter out existing IDs and shuffle the sequence to ensure randomness
    available_ids = [i for i in id_sequence if i not in existing_ids]
    np.random.shuffle(available_ids)
    
    # Take only as many IDs as there are NaN values
    unique_ids = available_ids[:num_nan]

    # Create a dictionary mapping NaN indices to unique IDs
    nan_indices = df[df[column_name].isnull()].index
    id_map = dict(zip(nan_indices, unique_ids))

    # Fill NaN values with the generated unique IDs
    df[column_name] = df[column_name].fillna(id_map)

    return df

df = generate_unique_ids_for_nan(df, 'User ID', 1000, 4999)

In [156]:
print(df['User ID'])

0       4174.0
2       1860.0
3       2294.0
4       2130.0
5       2095.0
         ...  
9994    1942.0
9995    1524.0
9996    4879.0
9997    2624.0
9999    4113.0
Name: User ID, Length: 9600, dtype: float64


In [157]:
# Let's convert User ID to int
df['User ID'] = df['User ID'].astype(int)

#### ==========================
#### 'Sleep Duration (hours)' column cleanup
#### ==========================

In [13]:
df['Sleep Duration (hours)'].unique()

array([7.16723562, 6.53823938,        nan, ..., 6.9165492 , 5.69123393,
       5.69100104])

In [152]:
# Replace empty strings,'ERROR' and NaN with 0
df['Sleep Duration (hours)'] = df['Sleep Duration (hours)'].replace(['', 'ERROR', np.nan], 0)

In [126]:
# Let's convert 'Sleep duration' to float
df['Sleep Duration (hours)'] = df['Sleep Duration (hours)'].astype(float)

In [127]:
# Looks like there is a negative value in Sleep Duration (hours) column, we'll replace it with 0
df['Sleep Duration (hours)'] = np.where(df['Sleep Duration (hours)'].lt(0), 0, df['Sleep Duration (hours)'])

#### ====================
#### 'Step Count' column cleanup
#### ====================

In [128]:
# Replace empty strings with 0
df['Step Count'] = df['Step Count'].replace([np.nan], 0)

In [129]:
# Let's convert 'Step Count' to int
df['Step Count'] = df['Step Count'].astype(int)

#### ====================
#### Round data in some columns
#### ====================

In [130]:
df = df.round({'Heart Rate (BPM)': 1, 'Blood Oxygen Level (%)': 1, 'Sleep Duration (hours)': 1})

In [131]:
# We can see that memory usage has decreased
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User ID                 9799 non-null   float64
 1   Heart Rate (BPM)        9600 non-null   float64
 2   Blood Oxygen Level (%)  9700 non-null   float64
 3   Step Count              10000 non-null  int32  
 4   Sleep Duration (hours)  9850 non-null   float64
 5   Activity Level          9800 non-null   object 
 6   Stress Level            9800 non-null   object 
dtypes: float64(4), int32(1), object(2)
memory usage: 507.9+ KB


#### ====================
#### 'Activity Level' column cleanup
#### ====================

In [134]:
# Let's fix some typos in Activity Level column
df['Activity Level'].unique()

array(['Highly_Active', 'Active', 'Sedentary', nan], dtype=object)

In [133]:
df['Activity Level'] = df['Activity Level'].replace('Seddentary', 'Sedentary')
df['Activity Level'] = df['Activity Level'].replace('Actve', 'Active')
df['Activity Level'] = df['Activity Level'].replace('Highly Active', 'Highly_Active')

In [137]:
# Let's see if there is correlation between 'Step Count' and 'Activity Level'
# Let's first find average number of steps in each activity level
print(df.groupby('Activity Level')['Step Count'].mean())

Activity Level
Active           6963.035528
Highly_Active    6888.711430
Sedentary        6909.849385
Name: Step Count, dtype: float64


In [143]:
# We can see that Activity Level doesn't correlate with step count
# Let's see if there is correlation between 'Heart Rate (BPM)' and 'Activity Level'
print(df.groupby('Activity Level')['Heart Rate (BPM)'].mean())

Activity Level
Active           75.969796
Highly_Active    76.093895
Sedentary        76.032110
Name: Heart Rate (BPM), dtype: float64


In [146]:
# To showcase one more strategy for filling missing values, let's populate missing Activity Level based on Step Count
df['Activity Level'] = np.where((df['Step Count'].lt(4000)) & (df['Activity Level'].isnull()), 'Sedentary', df['Activity Level'])
df['Activity Level'] = np.where((df['Step Count'].ge(4000) & df['Step Count'].lt(10000)) & (df['Activity Level'].isnull()), 'Active', df['Activity Level'])
df['Activity Level'] = np.where((df['Step Count'].ge(10000)) & (df['Activity Level'].isnull()), 'Highly_Active', df['Activity Level'])

#### ====================
#### 'Stress Level' column cleanup
#### ====================

In [139]:
df['Stress Level'].unique()

array(['1', '5', '3', '6', '10', '2', '8', '9', nan, '4', 'Very High',
       '7'], dtype=object)

In [140]:
# Let's fix some typos in Stress Level column
df['Stress Level'] = df['Stress Level'].replace('Very High', '10')

In [141]:
# Let's convert Stress Level column to numeric
df['Stress Level'] = pd.to_numeric(df['Stress Level'], errors='coerce')

In [58]:
# Let's replace missing values in 'Stress Level' column by median values
df['Stress Level'].fillna(df['Stress Level'].median(), inplace=True)

In [159]:
df['Stress Level'] = df['Stress Level'].astype(int)

#### =======================
#### 'Heart Rate (BPM)' column cleanup
#### =======================

In [145]:
# Let's assume that heart rate is the most important parameter in the dataset
# So we'll drop the rows that have missing values in that column
df = df.dropna(subset=['Heart Rate (BPM)'])

#### ====================
#### 'Blood Oxygen Level (%)' column cleanup
#### ====================

In [148]:
# Let's replace missing values in 'Blood Oxygen Level (%)' column by median values
df['Blood Oxygen Level (%)'].fillna(df['Blood Oxygen Level (%)'].median(), inplace=True)

In [160]:
# Let's look at the cleaned data
df.head(20)

,User ID,Heart Rate (BPM),Blood Oxygen Level (%),Step Count,Sleep Duration (hours),Activity Level,Stress Level
0,4174,58.9,98.8,5450,7.2,Highly_Active,1
2,1860,247.8,97.1,2826,0.0,Highly_Active,5
3,2294,40.0,96.9,13797,7.4,Active,3
4,2130,62.0,98.6,15679,0.0,Highly_Active,6
5,2095,96.3,94.2,10205,8.4,Highly_Active,10
6,4772,47.3,95.4,3208,7.9,Sedentary,2
7,4092,81.7,96.0,6051,5.2,Sedentary,1
8,3979,93.3,97.2,1775,5.5,Sedentary,1
9,3169,51.4,97.3,7940,8.0,Sedentary,8
10,1466,101.9,98.0,2987,7.5,Sedentary,9


In [117]:
df.to_csv('cleaned_smartwatch_health_data.csv', index=False)

In [162]:
# In this project we used different strategies for missing data replacement.